In [4]:
# Basic python utilities
import numpy as np
import scipy as scp 
from scipy.stats import gamma
from scipy.stats import mode
import pandas as pd

# Parallelization
import multiprocessing as mp
from  multiprocessing import Process
from  multiprocessing import Pool
import psutil

# System utilities
from datetime import datetime
import time
import os
import pickle 
import uuid

# My own code
import kde_class as kde
import ddm_data_simulation as ddm_simulator 
import boundary_functions as bf

# Plotting
import matplotlib.pyplot as plt

In [5]:
# Get the folder where base simulations live
folder_ = '/home/afengler/git_repos/nn_likelihoods/data_storage/kde/linear_collapse/base_simulations/'
files_ = os.listdir(folder_)
n_files = len(files_)
init_file = pickle.load(open( folder_ + files_[0], 'rb' ))
init_cols = list(init_file[2].keys())

In [6]:
# Initialize data frame
sim_stat_data = pd.DataFrame(np.zeros((n_files, len(init_cols))), columns = init_cols)

In [15]:
# MAX RT BY SIMULATION: TEST SHOULD BE CONSISTENT
n_simulations = init_file[2]['n_samples']
n_choices = len(init_file[2]['possible_choices'])

max_rts = []
max_ts = []

stds = np.zeros((n_files, n_choices))
mean_rts = np.zeros((n_files, n_choices))
choice_cnts = np.zeros((n_files, n_choices))
modes = np.zeros((n_files, n_choices))
mode_cnts = np.zeros((n_files, n_choices))

sim_stat_data = []

cnt = 0
for file_ in files_:
    tmp = pickle.load(open( folder_ + file_ , 'rb'))
    max_rts.append(tmp[0].max().round(2))
    max_ts.append(tmp[2]['max_t'])
    
    # Standard deviation of reaction times
    choice_cnt = 0
    for choice_tmp in tmp[2]['possible_choices']:
        
        tmp_rts = tmp[0][tmp[1] == choice_tmp]
        n_c = len(tmp_rts)
        choice_cnts[cnt, choice_cnt] = n_c
        
        mode_tmp = mode(tmp_rts)

        if n_c > 0:
            mean_rts[cnt, choice_cnt] = np.mean(tmp_rts)
            stds[cnt, choice_cnt] = np.std(tmp_rts)
            modes[cnt, choice_cnt] = float(mode_tmp[0])
            mode_cnts[cnt, choice_cnt] = int(mode_tmp[1])
        else:
            mean_rts[cnt, choice_cnt] = -1
            stds[cnt, choice_cnt] = -1
            modes[cnt, choice_cnt] = -1
            mode_cnts[cnt, choice_cnt] = 0
        
        choice_cnt += 1
    
    # Basic data column
    sim_stat_data.append([tmp[2][key] for key in list(tmp[2].keys())])
    
    cnt += 1
    if cnt % 1000 == 0:
        print(cnt)

sim_stat_data = pd.DataFrame(sim_stat_data, columns = init_file[2].keys())

for i in range(0, n_choices, 1):
    # Columns directly from stats collected in previous loop
    sim_stat_data['mean_rt_' + str(i)] = mean_rts[:, i]
    sim_stat_data['std_' + str(i)] = stds[:, i]
    sim_stat_data['choice_cnt_' + str(i)] = choice_cnts[:,i]
    sim_stat_data['mode_' + str(i)] = modes[:, i]
    sim_stat_data['mode_cnt_' + str(i)] = mode_cnts[:, i]
    
    # Derived Columns
    sim_stat_data['choice_prop_' + str(i)] = sim_stat_data['choice_cnt_' + str(i)] / n_simulations
    sim_stat_data['mode_cnt_rel_' + str(i)] = sim_stat_data['mode_cnt_' + str(i)] / sim_stat_data['choice_cnt_' + str(i)]
    
sim_stat_data['file'] = files_
        
# check that max_t is consistently the same value across simulations
assert len(np.unique(max_ts)) == 1

# check that max_rt is <= max_t + 0.00001 (adding for rounding)
assert max(max_rts) <= np.unique(max_ts)[0] + 0.0001

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000


In [ ]:
# SOME GRAPHS TO ILLUSTRATE BEHAVIOR OF SIMULATOR
grayscale_map = plt.get_cmap('seismic')
plt.figure()

plt.scatter(sim_stat_data['a'], sim_stat_data['mean_rt_1'],
            c = sim_stat_data['v'], 
            cmap = grayscale_map, 
            alpha = 0.1)

plt.colorbar()
plt.show()

In [ ]:
# SOME GRAPHS TO ILLUSTRATE BEHAVIOR OF SIMULATOR
grayscale_map = plt.get_cmap('seismic')
plt.figure()

plt.scatter(sim_stat_data['a'], sim_stat_data['mean_rt_0'],
            c = sim_stat_data['v'], 
            cmap = grayscale_map, 
            alpha = 0.3)

plt.colorbar()
plt.show()

In [ ]:
# VARIANCE RELATIONSHIP WITH A
# SOME GRAPHS TO ILLUSTRATE BEHAVIOR OF SIMULATOR
grayscale_map = plt.get_cmap('seismic')
plt.figure()

plt.scatter(sim_stat_data['a'], sim_stat_data['stds_0'],
            c = sim_stat_data['v'], 
            cmap = grayscale_map, 
            alpha = 0.3)

plt.colorbar()
plt.show()

In [ ]:
# VARIANCE RELATIONSHIP WITH A
# SOME GRAPHS TO ILLUSTRATE BEHAVIOR OF SIMULATOR
grayscale_map = plt.get_cmap('seismic')
plt.figure()

plt.scatter(sim_stat_data['a'], sim_stat_data['stds_1'],
            c = sim_stat_data['v'], 
            cmap = grayscale_map, 
            alpha = 0.3)

plt.colorbar()
plt.show()

In [ ]:
a = ['a', 'c', 'b']

In [ ]:
a.sort()

In [ ]:
list(sim_stat_data.keys())

In [ ]:
np.isclose(1e-8, 0)

In [ ]:
from scipy.stats import itemfreq
from scipy.stats import mode

In [ ]:
counts = np.unique(init_file[0], return_counts = True)

In [10]:
init_file[0]

array([[0.4 ],
       [0.29],
       [1.13],
       ...,
       [0.54],
       [0.37],
       [0.87]])

In [13]:
mode(init_file[0])[1]

array([[159]])

In [ ]:
int(res[0])

In [14]:
sim_stat_data

,v,a,w,s,node,theta,delta_t,max_t,n_samples,simulator,...,choice_prop_0,mode_cnt_rel_0,mean_rt_1,std_1,choice_cnt_1,mode_1,mode_cnt_1,choice_prop_1,mode_cnt_rel_1,file
0,2.151960,1.625399,0.527389,1,1.317172,1.168912,0.01,20,10000,ddm_flexbound,...,0.0015,0.120667,0.726888,0.344232,9985.0,0.44,0.44,0.9985,0.000044,ddm_flexbound_9aeb50be819411e99dd98f55dab4685c...
1,-0.618691,1.609516,0.372500,1,3.746727,1.043430,0.01,20,10000,ddm_flexbound,...,0.9320,0.000058,2.519338,1.314324,680.0,4.27,4.27,0.0680,0.006279,ddm_flexbound_7ad76bb2817811e9b33af5baff038307...
2,2.497972,2.420123,0.773865,1,2.250373,1.155046,0.01,20,10000,ddm_flexbound,...,0.0000,-inf,0.463350,0.271334,10000.0,0.32,0.32,1.0000,0.000032,ddm_flexbound_03b70888825111e987744d3f00e0d926...
3,0.760447,3.092238,0.190038,1,3.780733,0.548069,0.01,20,10000,ddm_flexbound,...,0.1947,0.000236,4.828464,1.333417,8053.0,5.17,5.17,0.8053,0.000642,ddm_flexbound_f1daf778825511e9b41839161ae7fdd0...
4,-0.467395,3.046099,0.810455,1,3.833527,1.562941,0.01,20,10000,ddm_flexbound,...,0.4436,0.000870,2.665162,1.397938,5564.0,3.86,3.86,0.5564,0.000694,ddm_flexbound_8f10589a820a11e98b16f99b5ecc4100...
5,-1.239418,3.176381,0.461665,1,0.377812,1.307336,0.01,20,10000,ddm_flexbound,...,0.9424,0.000087,1.088924,0.105193,576.0,1.17,1.17,0.0576,0.002031,ddm_flexbound_ba80b35e80bf11e9b234e9dca54d3a69...
6,-2.394653,2.622628,0.651032,1,3.978430,0.881549,0.01,20,10000,ddm_flexbound,...,0.9997,0.000124,1.190000,0.366697,3.0,0.78,0.78,0.0003,0.260000,ddm_flexbound_0a759f26817611e9851d730e125a1b26...
7,-0.533293,2.690001,0.238164,1,1.004148,0.676830,0.01,20,10000,ddm_flexbound,...,0.9675,0.000056,3.062800,0.576946,325.0,3.33,3.33,0.0325,0.010246,ddm_flexbound_61c30ce0808711e9af16a55095aa11fe...
8,1.038086,0.956857,0.461023,1,2.955403,0.567624,0.01,20,10000,ddm_flexbound,...,0.1328,0.000166,0.817499,0.583994,8672.0,0.36,0.36,0.8672,0.000042,ddm_flexbound_85cce286817311e9bc8d4f4b4427b74c...
9,-1.351886,1.436074,0.319850,1,3.244793,0.768687,0.01,20,10000,ddm_flexbound,...,0.9950,0.000030,1.523400,1.041016,50.0,0.69,0.69,0.0050,0.013800,ddm_flexbound_96c733f8825811e985fc85b9cb078a35...
